In [4]:
import os
import tarfile
from six.moves import urllib

EVASION_PATH = os.path.join("datasets", "evasao")


In [5]:
import pandas as pd

def load_evasion_data(evasion_path=EVASION_PATH):
    csv_path = os.path.join(evasion_path, "evasao.csv")
    return pd.read_csv(csv_path)

evasao = load_evasion_data()

In [9]:
evasao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2603 entries, 0 to 2602
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   curriculum_id                  2603 non-null   int64  
 1   student_id                     2603 non-null   int64  
 2   centro                         2603 non-null   object 
 3   department_name                2603 non-null   object 
 4   course_name                    2603 non-null   object 
 5   student_age_at_initial_period  2603 non-null   int64  
 6   birthday                       2603 non-null   object 
 7   birth_place                    2600 non-null   object 
 8   ingress_form                   2603 non-null   object 
 9   locks_number                   2603 non-null   int64  
 10  required_ch_progress           2603 non-null   float64
 11  optative_ch_progress           2603 non-null   float64
 12  curriculum_status              2603 non-null   o